In [0]:
import pandas as pd
import numpy as np
import datetime
import dateutil.parser

In [132]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
 df_fr = pd.read_csv('/content/drive/My Drive/SuperCase/data/df_full_flightreport.csv', sep=';')

In [0]:
 df_sg = pd.read_csv('/content/drive/My Drive/SuperCase/data/df_seatguru.csv')

In [0]:
df_ae = pd.read_csv('/content/drive/My Drive/SuperCase/data/data_airlinequality.csv')

In [0]:
columns_df_drop = [col for col in df_fr.columns if 'Unnamed' in col]
df_fr.drop(columns=columns_df_drop, inplace=True)

In [137]:
print(df_fr.columns)

Index(['id', 'Company', 'Flight n°', 'Class', 'Seat', 'Airplane', 'Duration',
       'Departure Date/Time', 'Arrival Date/Time', 'Departure City',
       'Arrival City', 'Company Grade', 'Cabin Grade', 'Cabin Crew Grade',
       'Entertainment/Wifi Grade', 'Meal/Catering Grade',
       'Lounge Global Grade', 'Lounge Comfort Grade',
       'Lounge Meal/Catering Grade', 'Lounge Entertainment/Wifi Grade',
       'Lounge Services Grade', 'Departure Airport Grade', 'Dep Efficiency',
       'Dep Access', 'Dep Services', 'Dep Cleanliness',
       'Arrival Airport Grade', 'Arr Efficiency', 'Arr Access', 'Arr Services',
       'Arr Cleanliness', 'Full Review', 'Conclusion/Summary'],
      dtype='object')


In [138]:
print(df_sg.columns)

Index(['Unnamed: 0', 'Date', 'Plane', 'Seat', 'Name', 'Comment', 'Url',
       'Category', 'Food', 'Wifi', 'Tv', 'Headphones', 'Elec', 'Sentiment',
       'Company'],
      dtype='object')


In [139]:
print(df_ae.columns)

Index(['Unnamed: 0', 'Aircraft', 'Cabin Staff Service', 'Date Flown',
       'Food & Beverages', 'Ground Service', 'Inflight Entertainment',
       'Recommended', 'Route', 'Seat Comfort', 'Seat Type',
       'Type Of Traveller', 'Unnamed: 0.1', 'Value For Money',
       'Wifi & Connectivity', 'company_name', 'country', 'date',
       'global Food & Beverages', 'global Inflight Entertainment',
       'global Seat Comfort', 'global Staff Service', 'global Value for Money',
       'review', 'title', 'verification'],
      dtype='object')


In [0]:
df_ta = pd.read_csv('/content/drive/My Drive/SuperCase/data/TripAdvisors_data/TripAdvisors_data_preprocessed.csv', sep='|')

In [141]:
print(df_ta.columns)


Index(['Unnamed: 0', 'username', 'entry_date', 'user_hometown',
       'user_contributions_number', 'user_useful_vote_number', 'flight_cities',
       'flight_type', 'passenger_class', 'general_grade', 'title',
       'review_text', 'travel_date', 'grades', 'helpful_review_vote_number',
       'company', 'user_homecountry', 'Legroom_grade', 'Seat comfort_grade',
       'In-flight Entertainment_grade', 'Customer service_grade',
       'Value for money_grade', 'Cleanliness_grade',
       'Check-in and boarding_grade', 'Food and Beverage_grade'],
      dtype='object')



# PreProcess

In [0]:
columns_to_keep = ['Cabin Staff Service', 'Food & Beverages', 'Ground Service',
                   'Inflight Entertainment', 'Seat Comfort', 'Value for Money',
                   'Class', 'Departure City', 'Arrival City', 'Aircraft', 'Date', 'User Country', 
                   'Review', 'Summary', 'Website']

### Airlinequality

In [0]:
rename_dict_ae = {
    'Seat Type' : 'Class',
    'Date Flown' : 'Date',
    'country' : 'User Country',
    'review' : 'Review',
    'title' : 'Summary',
    'Value For Money' : 'Value for Money'
}

processing

In [0]:
df_ae = pd.read_csv('/content/drive/My Drive/SuperCase/data/data_airlinequality.csv')

In [0]:
def parse_date(str_date):
  if str_date is np.nan:
    return np.nan
  else:
    return dateutil.parser.parse(str_date)

In [0]:
def get_dep(str_route):
  if str_route is np.nan:
    return [np.nan]
  else:
    return str_route.split('to')

def get_arr(str_route):
  if str_route is np.nan:
    return [np.nan]
  else:
    a = str_route.split('to')[-1]
    return a.split('via')

In [0]:
df_ae['Departure City'] = df_ae['Route'].apply(lambda a : get_dep(a)[0])

In [0]:
df_ae['Arrival City'] = df_ae['Route'].apply(lambda a : get_arr(a)[-1])

In [0]:
df_ae['Website'] = 'Skytrax'

In [0]:
df_ae.rename(columns=rename_dict_ae, inplace=True)

In [0]:
df_ae.Date = df_ae.Date.apply(parse_date)

In [0]:
df_ae[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']] = df_ae[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']].replace(['no','yes'], np.nan)

In [0]:
df_ae[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']] = df_ae[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']].astype(float, errors='ignore')

In [154]:
print(df_ae['Food & Beverages'].unique())
print(df_ae['Cabin Staff Service'].unique())
print(df_ae['Inflight Entertainment'].unique())
print(df_ae['Ground Service'].unique())
print(df_ae['Seat Comfort'].unique())

[nan  1.  3.  2.  5.  4.]
[ 1.  2.  4.  3.  5. nan]
[nan  1.  4.  5.  3.  2.]
[ 1.  5.  4.  3.  2. nan]
[ 1.  4.  2.  3.  5. nan]


In [155]:
df_ae.shape

(21720, 29)

In [0]:
df_ae_uniform = df_ae[columns_to_keep]

### FlighReport

In [0]:
rename_dict_fr = {
    'Cabin Crew Grade' : 'Cabin Staff Service',
    'Meal/Catering Grade' : 'Food & Beverages',
    'Departure Airport Grade' : 'Ground Service',
    'Entertainment/Wifi Grade' : 'Inflight Entertainment' ,
    'Cabin Grade' : 'Seat Comfort',
    'Airplane' : 'Aircraft',
    'Departure Date/Time' : 'Date',
    'Full Review' : 'Review',
    'Conclusion/Summary' : 'Summary'
}

In [0]:
 df_fr = pd.read_csv('/content/drive/My Drive/SuperCase/data/df_full_flightreport.csv', sep=';')

In [0]:
df_fr['Website'] = 'Flightreport'

In [0]:
df_fr.rename(columns=rename_dict_fr, inplace=True)

In [0]:
def get_mark(str_):
  if type(str_) == str:
    mark = str_.split('/')
    return(float(mark[0])/2)
  else:
    return np.nan

In [0]:
df_fr['Ground Service'] = df_fr['Ground Service'].apply(lambda a : get_mark(a))

In [0]:
df_fr.Date = df_fr.Date.apply(parse_date)

In [0]:
df_fr[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']] = df_fr[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']].astype(float, errors='ignore')

In [0]:
df_fr[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']] = df_fr[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']]/2

In [0]:
columns_to_keep_fr = ['Cabin Staff Service', 'Food & Beverages', 'Ground Service',
                   'Inflight Entertainment', 'Seat Comfort',
                   'Class', 'Departure City', 'Arrival City', 'Aircraft', 'Date', 
                   'Review', 'Summary', 'Website']

In [167]:
print(df_fr['Food & Beverages'].unique())
print(df_fr['Cabin Staff Service'].unique())
print(df_fr['Inflight Entertainment'].unique())
print(df_fr['Ground Service'].unique())
print(df_fr['Seat Comfort'].unique())

[4.5  3.   3.25 5.   4.   1.5  4.25 2.5   nan 4.75 3.5  3.75 2.   2.25
 2.75 0.25 1.25 0.   1.   0.5  1.75 0.75]
[5.   4.25 4.   4.5  2.5  3.5  3.75 2.   4.75 3.25 1.5  0.5  2.25 3.
 2.75 0.75 1.   1.25  nan 1.75 0.25 0.  ]
[5.   3.   2.5  4.5  4.25 4.75 4.   2.75 3.5  2.25 3.75 2.   1.   3.25
 0.   1.25 1.5  0.5  1.75 0.75  nan 0.25]
[2.    1.9   2.5   2.45  2.325 0.975 2.025 1.875 1.5   2.375 2.05  1.575
 1.75  2.2   2.275 1.85  2.15  2.125 1.7   1.725 2.3   1.625 2.25  1.775
 1.375 1.8   1.25  1.95  1.975 2.1   2.35  1.    2.225 0.875 1.65  2.4
 1.6   1.525 0.75  1.275 1.35  0.95  1.55  0.85  1.2   1.1   0.7   2.075
 1.3   0.525 1.475 1.05  0.6   0.9   1.4   1.45  1.15  1.825 0.625 1.125
 2.475 0.65    nan 1.325 1.025 0.45  0.775 0.8   0.825 1.075 1.225 0.375
 0.575 0.125 0.4   0.725]
[4.   3.75 2.25 4.5  5.   3.5  4.75 1.5  4.25 2.   2.5  2.75 3.   0.5
 3.25 1.   0.25 1.25 1.75 0.    nan 0.75]


In [0]:
df_fr_uniform = df_fr[columns_to_keep_fr]

### TripAdvisor

In [0]:
rename_dict_ta = {
    'Customer service_grade' : 'Cabin Staff Service',
    'Food and Beverage_grade' : 'Food & Beverages',
    'Check-in and boarding_grade' : 'Ground Service',
    'In-flight Entertainment_grade' : 'Inflight Entertainment' ,
    'Value for money_grade' : 'Value for Money',
    'passenger_class' : 'Class',
    'Cabin Grade' : 'Seat Comfort',
    'Airplane' : 'Aircraft',
    'travel_date' : 'Date',
    'user_hometown' : 'User Country',
    'review_text' : 'Review',
    'title' : 'Summary'
}

In [0]:
df_ta = pd.read_csv('/content/drive/My Drive/SuperCase/data/TripAdvisors_data/TripAdvisors_data_preprocessed.csv', sep='|')

In [0]:
df_ta.rename(columns=rename_dict_ta, inplace=True)

In [0]:
df_ta.Date = df_ta.Date.apply(parse_date)

In [0]:
df_ta['Seat Comfort'] = df_ta['Legroom_grade'] + df_ta['Seat comfort_grade'] / 2

In [0]:
df_ta[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']] = df_ta[['Food & Beverages', 'Cabin Staff Service', 'Inflight Entertainment', 'Ground Service', 'Seat Comfort']].astype(float, errors='ignore')

In [0]:
df_ta['Seat Comfort'] = (df_ta['Legroom_grade'] + df_ta['Seat comfort_grade']) / 2

In [0]:
def get_dep_arr_ta(str_route):
  if str_route is np.nan:
    return [np.nan]
  else:
    return str_route.split('-')

In [0]:
df_ta['Departure City'] = df_ta.flight_cities.apply(lambda a : get_dep_arr_ta(a)[0])

In [0]:
df_ta['Arrival City'] = df_ta.flight_cities.apply(lambda a : get_dep_arr_ta(a)[-1])

In [0]:
df_ta['Website'] = 'Tripadvisor'

In [0]:
columns_to_keep_ta = ['Cabin Staff Service', 'Food & Beverages', 'Ground Service',
                   'Inflight Entertainment', 'Seat Comfort', 'Value for Money',
                   'Class', 'Departure City', 'Arrival City', 'Date', 'User Country', 
                   'Review', 'Summary', 'Website']

In [0]:
df_ta_uniform = df_ta[columns_to_keep_ta]

# MERGING

In [0]:
df_ae_uniform = df_ae[columns_to_keep]

In [0]:
df_ta_uniform = df_ta[columns_to_keep_ta]

In [0]:
df_fr_uniform = df_fr[columns_to_keep_fr]

In [0]:
def clean_airplanes(str_ariplane):
  if type(str_ariplane) is str:
    if '/' in str_ariplane:
      return str_ariplane.split('/')[0]
    elif '-' in str_ariplane:
      return str_ariplane.split('-')[0]
    elif 'and' in str_ariplane:
      return str_ariplane.split('and')[0]
    elif '&' in str_ariplane:
      return str_ariplane.split('&')[0]
    elif ',' in str_ariplane:
      return str_ariplane.split(',')[0]
    else:
      return str_ariplane
  else:
    return str_ariplane  

In [0]:
def uniformize_aircraft(str_aircraft):
  if type(str_aircraft) is str:
    a = str_aircraft.strip().replace(' ','')
    if str.startswith(str_aircraft, 'A') and len(a) == 4:
      return 'Airbus' + a
    elif str.startswith(str_aircraft, 'B') and len(a) == 4:
      return 'Boeing' + a[1:]
    else:
      return a
  else:
    return str_aircraft
  

In [187]:
df_ae_uniform['Aircraft'] = df_ae_uniform['Aircraft'].apply(lambda a : clean_airplanes(a))
df_ae_uniform['Aircraft'] = df_ae_uniform['Aircraft'].apply(lambda a : clean_airplanes(a))
df_ae_uniform['Aircraft'] = df_ae_uniform.Aircraft.apply(lambda a : uniformize_aircraft(a))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [188]:
df_fr_uniform['Aircraft'] = df_fr_uniform['Aircraft'].apply(lambda a : clean_airplanes(a))
df_fr_uniform['Aircraft'] = df_fr_uniform['Aircraft'].apply(lambda a : clean_airplanes(a))
df_fr_uniform['Aircraft'] = df_fr_uniform.Aircraft.apply(lambda a : uniformize_aircraft(a))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [0]:
#df_ae_uniform.loc[:,'Class'] = df_ae_uniform.loc[:,'Class'].replace('Premium Economy', 'Economy Class')

In [0]:
l_change_fr = [['Economy', 'Domestic First', 'Business', 'Premium Eco', 'First'], 
               ['Economy Class', 'First Class', 'Business Class', 'Premium Economy', 'First Class']]

In [191]:
df_fr_uniform.loc[:,'Class'] = df_fr_uniform.loc[:,'Class'].replace(['Economy', 'Domestic First', 'Business', 'Premium Eco', 'First'], 
                                                                    ['Economy Class', 'First Class', 'Business Class', 'Premium Economy', 'First Class'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [192]:
df_ta_uniform.loc[:,'Class'] = df_ta_uniform.loc[:,'Class'].replace('Economy', 'Economy Class')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
df_merged = pd.concat([df_ae_uniform, df_fr_uniform, df_ta_uniform], sort=False)

In [194]:
max(df_merged['Seat Comfort'])

5.0

In [0]:
df_merged.to_csv('data_ae_fr_ta_2.csv')

In [199]:
df_merged.columns

Index(['Cabin Staff Service', 'Food & Beverages', 'Ground Service',
       'Inflight Entertainment', 'Seat Comfort', 'Value for Money', 'Class',
       'Departure City', 'Arrival City', 'Aircraft', 'Date', 'User Country',
       'Review', 'Summary', 'Website'],
      dtype='object')

In [196]:
df_merged.head(2)

,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Seat Comfort,Value for Money,Class,Departure City,Arrival City,Aircraft,Date,User Country,Review,Summary,Website
0,1.0,NaN,1.0,NaN,1.0,1,Economy Class,Frankfurt,Pristina,NaN,2019-09-29 00:00:00,Serbia,Please do a favor yourself and do not fly wit...,I will never fly again with Adria,Skytrax
1,1.0,1.0,1.0,1.0,1.0,1,Economy Class,Sofia,Ljubljana,NaN,2019-09-29 00:00:00,Netherlands,Do not book a flight with this airline! My fri...,it ruined our last days of holidays,Skytrax
